In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

from copy import deepcopy


#For random forest for time series:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

#For metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel
from Logic import logic_bat, logic_DP, logic_actions
from Funcs_Logic_DP import (get_price, logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, 
                            logic_series_print, DP_stochastic, policy_rollout)

In [2]:
house = "h16"
prod = load_series("p",house,"prod_"+house,[24,48,168])
cons = load_series("c",house,"cons_"+house,[24,48,168])
cons

,cons_h16,cons_h16-24,cons_h16-48,cons_h16-168,Year,Month,Day,Weekday,Hour
2020-12-29 00:00:00,5.7,5.7,6.9,6.3,2020,12,29,1,0
2020-12-29 01:00:00,5.9,5.4,5.3,4.8,2020,12,29,1,1
2020-12-29 02:00:00,5.0,6.0,4.9,4.8,2020,12,29,1,2
2020-12-29 03:00:00,3.9,6.8,4.4,4.3,2020,12,29,1,3
2020-12-29 04:00:00,3.5,4.0,3.6,4.0,2020,12,29,1,4
...,...,...,...,...,...,...,...,...,...
2023-01-10 16:00:00,6.1,7.6,6.9,5.4,2023,1,10,1,16
2023-01-10 17:00:00,8.3,10.0,9.8,6.2,2023,1,10,1,17
2023-01-10 18:00:00,10.4,9.1,14.5,13.0,2023,1,10,1,18
2023-01-10 19:00:00,9.9,11.1,10.8,10.4,2023,1,10,1,19


In [3]:
merged = merge(house)
merged

,prod_h16,cons_h16,power_yield,SpotPriceDKK
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001
...,...,...,...,...
2023-01-09 16:00:00,0.1,7.6,-7.5,1204.510010
2023-01-09 17:00:00,0.0,10.0,-10.0,1296.579956
2023-01-09 18:00:00,0.0,9.1,-9.1,1354.890015
2023-01-09 19:00:00,0.0,11.1,-11.1,1266.089966


In [4]:
N = 24*7
bat_capacity=13
Start="2022-06-19 00:00:00"
End = pd.date_range(start=Start, periods=N, freq="h")[-1]

train_prod = 2*7*24
train_cons = 3000

BeforeProd = pd.date_range(end=Start, periods=train_prod+1, freq="h")[0]
BeforeCons = pd.date_range(end=Start, periods=train_cons+1, freq="h")[0]

In [5]:
forecasterProd = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=8,max_features=3,n_estimators=100,n_jobs=4),
                 lags      = list(range(1,10+1,1))
                )
forecasterCons = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=8,max_features=4,n_estimators=100,n_jobs=4),
                 lags      = list(range(1,10+1,1))
                )

battery_no = Battery(max_capacity=bat_capacity,max_charge=0)
battery = Battery(max_capacity=bat_capacity,max_charge=7)
DPbat  = Battery(max_capacity=bat_capacity,max_charge=7)

series_no_battery = pd.DataFrame(columns=merged.columns)
series_battery    = pd.DataFrame(columns=merged.columns)
series_battery_DP = pd.DataFrame(columns=merged.columns)

preds = pd.DataFrame()

all_preds = []
stepsize=24 
for i in range(N//stepsize):#stepsize has to be even
    _, predProd = backtesting_forecaster(forecaster = forecasterProd,
                                         y          = prod.loc[BeforeProd:End].iloc[i*stepsize:train_prod+i*stepsize+24]['prod_'+house],
                                         exog       = prod.loc[BeforeProd:End].iloc[i*stepsize:train_prod+i*stepsize+24].drop(['prod_'+house], axis=1),
                                         initial_train_size = train_prod,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )

    _, predCons = backtesting_forecaster(forecaster = forecasterCons,
                                         y          = cons.loc[BeforeCons:End].iloc[i*stepsize:train_cons+i*stepsize+24]['cons_'+house],
                                         exog       = cons.loc[BeforeCons:End].iloc[i*stepsize:train_cons+i*stepsize+24].drop(['cons_'+house], axis=1),
                                         initial_train_size = train_cons,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )
    pred_i = pd.merge(predProd, predCons, how="outer", left_index=True,right_index=True)
    pred_i = pred_i.rename(columns={"pred_x":"prod_"+house,"pred_y":"cons_"+house})

    pred_i["prod_"+house]=np.array((pred_i["prod_"+house].to_numpy())*10,dtype=int)/10
    pred_i["cons_"+house]=np.array((pred_i["cons_"+house].to_numpy())*10,dtype=int)/10

    pred_i["power_yield"] = pred_i["prod_"+house] - pred_i["cons_"+house]
    pred_i["SpotPriceDKK"] = merged.loc[pred_i.index[0]:pred_i.index[-1]]["SpotPriceDKK"]
    
    merged_i = merged.loc[Start:End].iloc[i*stepsize:i*stepsize+24]
    
    temp_no_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery_no), logic_bat)
    temp_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery), logic_bat)

    DP = DPModel(24, pred_i.index[0], pred_i.index[-1], house, pred_i, deepcopy(DPbat))
    _, pi = DP_stochastic(DP)
    print(DPbat.get_current_capacity())
    _, _, actions = policy_rollout(DP,pi=lambda x, k: pi[k][x],x0=DPbat.get_current_capacity())
    print(actions)
    
    series_no_battery_i = action_rollout(merged_i.iloc[:stepsize], battery_no, temp_no_bat[:stepsize])
    series_battery_i    = action_rollout(merged_i.iloc[:stepsize], battery, temp_bat[:stepsize])
    series_battery_DP_i = action_rollout(merged_i.iloc[:stepsize], DPbat, actions)
    
    
    preds=pd.concat([preds,pred_i[:stepsize]])
    series_no_battery = pd.concat([series_no_battery,series_no_battery_i])
    series_battery    = pd.concat([series_battery,series_battery_i])
    series_battery_DP = pd.concat([series_battery_DP,series_battery_DP_i])

series_no_battery["price_cummulative"] = series_no_battery["price"].cumsum(axis=0)
series_battery["price_cummulative"] = series_battery["price"].cumsum(axis=0) 
series_battery_DP["price_cummulative"] = series_battery_DP["price"].cumsum(axis=0) 
preds

0  0
                     charge  buy
2022-06-19 00:00:00    -0.0  0.0
2022-06-19 01:00:00    -0.0  0.0
2022-06-19 02:00:00    -0.0  0.0
2022-06-19 03:00:00    -0.0  0.0
2022-06-19 04:00:00    -0.0  0.0
2022-06-19 05:00:00    -0.0  0.0
2022-06-19 06:00:00    -0.0  0.0
2022-06-19 07:00:00    -0.0  0.0
2022-06-19 08:00:00    -0.0  0.0
2022-06-19 09:00:00    -0.0  0.0
2022-06-19 10:00:00    -0.0  0.0
2022-06-19 11:00:00    -0.0  0.0
2022-06-19 12:00:00    -0.0  0.0
2022-06-19 13:00:00    -0.0  0.0
2022-06-19 14:00:00     7.0  0.0
2022-06-19 15:00:00     6.4  0.0
2022-06-19 16:00:00     0.7  0.0
2022-06-19 17:00:00     0.7  0.0
2022-06-19 18:00:00     0.7  0.0
2022-06-19 19:00:00    -0.3  0.0
2022-06-19 20:00:00    -3.4  0.0
2022-06-19 21:00:00    -5.6  0.0
2022-06-19 22:00:00    -1.9  0.0
2022-06-19 23:00:00    -0.0  0.0
0  0.0
                     charge  buy
2022-06-20 00:00:00    -0.0  0.0
2022-06-20 01:00:00    -0.0  0.0
2022-06-20 02:00:00    -0.0  0.0
2022-06-20 03:00:00     6.0  6.

,prod_h16,cons_h16,power_yield,SpotPriceDKK
2022-06-19 00:00:00,0.0,6.3,-6.3,1540.390015
2022-06-19 01:00:00,0.0,5.7,-5.7,1413.410034
2022-06-19 02:00:00,0.0,4.7,-4.7,1231.300049
2022-06-19 03:00:00,0.0,4.0,-4.0,1039.969971
2022-06-19 04:00:00,0.0,3.7,-3.7,897.140015
...,...,...,...,...
2022-06-25 19:00:00,9.4,10.8,-1.4,2336.860107
2022-06-25 20:00:00,5.3,9.4,-4.1,2434.100098
2022-06-25 21:00:00,1.6,8.5,-6.9,2400.699951
2022-06-25 22:00:00,0.2,7.4,-7.2,2410.669922


In [6]:
print_price_summary(series_no_battery)
logic_series_print(series_no_battery)

Cost for period: 2022-06-19 00:00:00 to 2022-06-25 23:00:00 is:  802.0  DKK
Average cost per year is: 42077.0 DKK
Number of kwh purchased in the period: 456.29999999990105
Average number of kwh purchased per year: 23951.651497000792
Average number of kwh sold per year: 34334.37485029512
hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   1.5404, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721
    1:   1.4134, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632
    2:   1.2313, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347
    3:   1.0400, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866
    4:   0.8971, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.4091, 28.4957
    5:   0.8254, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  2.8065, 31.3022
    6:   0.8373, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  2.9304, 34.2326
    7:

In [7]:
print_price_summary(series_battery)
logic_series_print(series_battery)

Cost for period: 2022-06-19 00:00:00 to 2022-06-25 23:00:00 is:  746.0  DKK
Average cost per year is: 39160.0 DKK
Number of kwh purchased in the period: 446.7999999997963
Average number of kwh purchased per year: 23452.98682633661
Average number of kwh sold per year: 31436.87065867286
hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   1.5404, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721
    1:   1.4134, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632
    2:   1.2313, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347
    3:   1.0400, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866
    4:   0.8971, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.4091, 28.4957
    5:   0.8254, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  2.8065, 31.3022
    6:   0.8373, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  2.9304, 34.2326
    7:  

   92:   3.4436,-10.4000, -8.2000,  0.0000, -2.2000,  7.3000,  5.1000, 28.2372,446.4281
   93:   2.9704, -9.1000, -4.3000,  0.0000, -4.8000,  4.8000,  0.0000, 12.7726,459.2007
   94:   2.7863, -8.1000, -8.1000,  0.0000,  0.0000,  0.0000,  0.0000, 22.5687,481.7694
   95:   2.2286, -7.7000, -7.7000,  0.0000,  0.0000,  0.0000,  0.0000, 17.1599,498.9293
   96:   2.1469, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000, 11.3784,510.3077
   97:   1.9186, -4.5000, -4.5000,  0.0000,  0.0000,  0.0000,  0.0000,  8.6338,518.9415
   98:   1.8606, -4.3000, -4.3000,  0.0000,  0.0000,  0.0000,  0.0000,  8.0006,526.9421
   99:   1.8287, -3.6000, -3.6000,  0.0000,  0.0000,  0.0000,  0.0000,  6.5832,533.5253
  100:   1.8976, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  6.4517,539.9770
  101:   2.0806, -3.6000, -3.6000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4901,547.4671
  102:   2.5003, -2.9000, -2.9000,  0.0000,  0.0000,  0.0000,  0.0000,  7.2508,554.7179
  103:   2.7030, -2.4000, -2.400

In [8]:
print_price_summary(series_battery_DP)
logic_series_print(series_battery_DP)

Cost for period: 2022-06-19 00:00:00 to 2022-06-25 23:00:00 is:  710.0  DKK
Average cost per year is: 37269.0 DKK
Number of kwh purchased in the period: 450.80000000002326
Average number of kwh purchased per year: 23662.950898204814
Average number of kwh sold per year: 32334.46706587039
hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   1.5404, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  8.4721,  8.4721
    1:   1.4134, -5.3000, -5.3000,  0.0000, -0.0000,  0.0000,  0.0000,  7.4911, 15.9632
    2:   1.2313, -4.2000, -4.2000,  0.0000, -0.0000,  0.0000,  0.0000,  5.1715, 21.1347
    3:   1.0400, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  3.9519, 25.0866
    4:   0.8971, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  3.4091, 28.4957
    5:   0.8254, -3.4000, -3.4000,  0.0000, -0.0000,  0.0000,  0.0000,  2.8065, 31.3022
    6:   0.8373, -3.5000, -3.5000,  0.0000, -0.0000,  0.0000,  0.0000,  2.9304, 34.2326
    7:

  157:   1.3424, 19.3000, 19.3000,  0.0000, -0.0000,  0.0000,  0.0000, -2.5908,677.2216
  158:   1.2206, 18.7000, 11.7000,  0.0000,  7.0000,  0.0000,  7.0000, -1.4281,675.7935
  159:   1.3759, 21.3000, 14.9000,  0.0000,  6.4000,  6.6000, 13.0000, -2.0501,673.7434
  160:   1.6417, 17.2000, 16.5000,  0.0000,  0.7000, 12.3000, 13.0000, -2.7088,671.0346
  161:   2.0018, 14.7000, 17.0000,  0.0000, -2.3000, 12.3000, 10.0000, -3.4031,667.6315
  162:   2.0204,  0.4000, -3.1000,  3.1000,  3.5000,  9.5000, 13.0000,  6.2634,673.8949
  163:   2.3369, -1.8000, -1.5000,  0.0000, -0.3000, 12.3000, 12.0000,  3.5053,677.4002
  164:   2.4341, -5.0000, -0.9000,  0.0000, -4.1000, 11.4000,  7.3000,  2.1907,679.5908
  165:   2.4007, -6.2000,  0.7000,  0.0000, -6.9000,  6.9000,  0.0000, -0.1680,679.4228
  166:   2.4107, -6.5000, -6.5000,  0.0000, -0.0000,  0.0000,  0.0000, 15.6694,695.0921
  167:   2.1943, -6.8000, -6.8000,  0.0000, -0.0000,  0.0000,  0.0000, 14.9214,710.0135
